In [1]:
# Importing the necessary packages

from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
page = requests.get('https://www.politifact.com/factchecks/list/?page=1') # downloads the pge
soup = BeautifulSoup(page.content, 'html.parser') # for parsing the downloaded page

In [3]:
df = pd.DataFrame()
# columns = ['Statement', 'Link', 'Date', 'Source', 'Label']
df

""


In [4]:
prefix = 'https://www.politifact.com'

results = soup.find('ul', class_ = "o-listicle__list")
for post in results.find_all('article'):
    source = post.find('a').get_text().strip() # this is just one source from one post REM: for post in...
    date = post.find('footer', class_ = "m-statement__footer").get_text('•').split('•')[1]
    statement = post.find('div', class_ = "m-statement__quote").get_text().strip()
    label = post.find_all('picture')[1].img['alt']
    link = prefix + [a['href'] for a in post.find_all('a', href = True)][1]
    
    data = {
        'Statement': [statement],
        'Link': [link],
        'Date': [date],
        'Source': [source],
        'Label': [label]
    }
    temp_df = pd.DataFrame(data)
    df = pd.concat([df, temp_df], ignore_index = True)
   

df.to_excel('politifact headlines.xlsx')

In [5]:
# modifying the code a little bit to be able to pick data from the first 10 pages

for i in range(1, 11):
    url = 'https://www.politifact.com/factchecks/list/?page={}'.format(i)
    page = requests.get(url) # downloads the pge
    soup = BeautifulSoup(page.content, 'html.parser') # for parsing the downloaded page
    
    prefix = 'https://www.politifact.com'

    results = soup.find('ul', class_ = "o-listicle__list")
    for post in results.find_all('article'):
        source = post.find('a').get_text().strip() # this is just one source from one post REM: for post in...
        date = ' '.join(post.find('footer', class_ = "m-statement__footer").get_text().strip('\n').split()[-3:])
        statement = post.find('div', class_ = "m-statement__quote").get_text().strip()
        label = post.find_all('picture')[1].img['alt']
        link = prefix + [a['href'] for a in post.find_all('a', href = True)][1]

        data = {
            'Statement': [statement],
            'Link': [link],
            'Date': date,
            'Source': [source],
            'Label': [label]
        }
        temp_df = pd.DataFrame(data)
        df = pd.concat([df, temp_df], ignore_index = True)

df

,Statement,Link,Date,Source,Label
0,H.R. 1 “would allow the Boston bomber to have ...,https://www.politifact.com/factchecks/2021/mar...,"March 5, 2021\n",Charlie Kirk,false
1,Video shows someone wearing an Anthony Fauci m...,https://www.politifact.com/factchecks/2021/mar...,"March 5, 2021\n",Viral image,pants-fire
2,"""Youth usage of marijuana has actually gone do...",https://www.politifact.com/factchecks/2021/mar...,"March 5, 2021\n",Melissa Agard,half-true
3,“The National Academy of Medicine points to a ...,https://www.politifact.com/factchecks/2021/mar...,"March 5, 2021\n",Chris Kapenga,true
4,Says “Biden took down Dr. Seuss and Mr. Potato...,https://www.politifact.com/factchecks/2021/mar...,"March 5, 2021\n",Facebook posts,false
...,...,...,...,...,...
325,The woman killed during the Jan. 6 U.S. Capito...,https://www.politifact.com/factchecks/2021/jan...,"January 7, 2021",Jonathan Brostoff,false
326,The storming of the Capitol was staged.,https://www.politifact.com/factchecks/2021/jan...,"January 7, 2021",Instagram posts,false
327,Quotes show Democrats supported riots “when BL...,https://www.politifact.com/factchecks/2021/jan...,"January 7, 2021",Facebook posts,barely-true
328,“Evidence growing that fascist ANTIFA orchestr...,https://www.politifact.com/factchecks/2021/jan...,"January 7, 2021",Mo Brooks,pants-fire
